In [2]:
pip install faker pandas numpy 

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pandas as pd

# Folder path where the CSV files are stored
folder_path = '/workspace/Datafest/data'

# Iterate through every file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Print the file name and the data types of each column
        print(f"\nData types for file: {filename}")
        print(df.dtypes)



Data types for file: attendance_table.csv
Student_ID        object
Days_Attended      int64
Days_Missed        int64
Absence_Reason    object
dtype: object

Data types for file: class_resources_table.csv
Class_ID                         object
Number_of_Students                int64
Number_of_Teachers                int64
Weekly_Teaching_Hours             int64
Weekly_Library_Time               int64
Weekly_Computer_Training_Time     int64
Weekly_Lab_Hours                  int64
Chalkboard                        int64
Basic_Textbooks                   int64
Chairs_Desks                      int64
Functional_Fans                   int64
dtype: object

Data types for file: extracurricular_activity.csv
Student_ID                  object
Extracurricular_Activity    object
Weekly_Hours                 int64
dtype: object

Data types for file: parent_table.csv
Student_ID                       object
Fathers Name                     object
Mothers Name                     object
Family Name 

In [6]:
import psycopg2
import os
import pandas as pd
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Access the variables

database_url = os.getenv("DATABASE_URL")


# Connect to PostgreSQL database
conn = psycopg2.connect(database_url)

# Create a cursor object
cursor = conn.cursor()

# Get all table names from the current schema
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public'
""")
tables = cursor.fetchall()

# Create a dictionary to hold the data from each table
table_data = {}

# Loop through each table and load the data into a variable (as a pandas DataFrame)
for table in tables:
    table_name = table[0]
    query = f"SELECT * FROM {table_name}"
    table_df = pd.read_sql(query, conn)
    
    # Store each table's data as a DataFrame in the dictionary, with the table name as the key
    table_data[table_name] = table_df

# Close the connection
cursor.close()
conn.close()

# Access data of each table from `table_data` dictionary
# Example: To access the 'student_table' data
student_data = table_data['student_table']
attendance_table = table_data['attendance_table']
class_resources_table = table_data['class_resources_table']
extracurricular_activity = table_data['extracurricular_activity']
parent_table = table_data['parent_table']
ss3_student_survey = table_data['ss3_student_survey']
staff_table = table_data['staff_table']
student_performance = table_data['student_performance']
teachers_table = table_data['teachers_table']



/tmp/ipykernel_884/1478016514.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_df = pd.read_sql(query, conn)


In [4]:
from faker import Faker
import numpy as np
fake = Faker()
Faker.seed(42)
np.random.seed(42)

fake.unique.uuid4().replace('-','')


'bdd640fb06674ad19c80317fa3b1799d'

In [11]:
student_data.class_id.unique()

array(['SS1 Class D', 'SS1 Class C', 'SS3 Class F', 'SS1 Class E',
       'SS2 Class E', 'SS2 Class A', 'SS2 Class B', 'SS3 Class A',
       'SS1 Class B', 'SS2 Class D', 'SS3 Class C', 'SS1 Class F',
       'SS3 Class D', 'SS3 Class B', 'SS3 Class E', 'SS2 Class C',
       'SS1 Class A', 'SS2 Class F'], dtype=object)

In [3]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_behindthename(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.browsename')
    return [name.text.strip() for name in names]

def scrape_momjunction(base_url, gender):
    all_names = []
    page = 1
    while True:
        url = f"{base_url}?gender={gender}&page={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': f'baby-name-{gender}'})
        
        if not table:
            break
        
        rows = table.find_all('tr')[1:]  # Skip header row
        names = [row.find_all('td')[1].text.strip() for row in rows]
        
        if not names:
            break
        
        all_names.extend(names)
        page += 1
    
    return all_names


female_names = scrape_behindthename('https://www.behindthename.com/names/gender/feminine/usage/nigerian')
male_names = scrape_behindthename('https://www.behindthename.com/names/gender/masculine/usage/nigerian')
unisex_names = scrape_behindthename('https://www.behindthename.com/names/gender/unisex/usage/nigerian')

momjunction_url = 'https://www.momjunction.com/baby-names/nigerian/'
female_names.extend(scrape_momjunction(momjunction_url, 'girl'))
male_names.extend(scrape_momjunction(momjunction_url, 'boy'))

# Remove duplicates and sort
female_names = sorted(set(female_names))
male_names = sorted(set(male_names))
unisex_names = sorted(set(unisex_names))

def parse_name_data(name_data):
    parts = name_data.split(' ')
    first_name = parts[0]  
    ethnic_group = (
        'Hausa' if 'Hausa' in name_data else
        'Yoruba' if 'Yoruba' in name_data else
        'Igbo' if 'Igbo' in name_data else
        'Urhobo' if 'Urhobo' in name_data else
        'Ibibio' if 'Ibibio' in name_data else
        'Other'
    )
    return first_name, ethnic_group

# Create dictionary by parsing each entry
name_ethnic_male = {parse_name_data(name)[0]: parse_name_data(name)[1] for name in male_names}
name_ethnic_female = {parse_name_data(name)[0]: parse_name_data(name)[1] for name in female_names}
name_ethnic_female


{'Abeni': 'Yoruba',
 'Abidemi': 'Yoruba',
 'Abimbola': 'Yoruba',
 'Abiodun': 'Yoruba',
 'Abiola': 'Yoruba',
 'Abioye': 'Yoruba',
 'Abosede': 'Yoruba',
 'Adaeze': 'Igbo',
 'Adanna': 'Igbo',
 'Adannaya': 'Igbo',
 'Ade': 'Yoruba',
 'Adebola': 'Yoruba',
 'Adebowale': 'Yoruba',
 'Adedayo': 'Yoruba',
 'Adenike': 'Yoruba',
 'Adeola': 'Yoruba',
 'Adetokunbo': 'Yoruba',
 'Adisa': 'Yoruba',
 'Aisha': 'Hausa',
 'Aishatu': 'Hausa',
 'Akachi': 'Igbo',
 'Akpofure': 'Urhobo',
 'Akuchi': 'Igbo',
 'Alaba': 'Yoruba',
 'Alheri': 'Hausa',
 'Amaka': 'Igbo',
 'Amara': 'Igbo',
 'Amarachi': 'Igbo',
 'Amina': 'Hausa',
 'Anuli': 'Igbo',
 'Asabe': 'Hausa',
 "Asma'u": 'Hausa',
 'Ayo': 'Yoruba',
 'Ayodele': 'Yoruba',
 'Ayomide': 'Yoruba',
 'Ayotunde': 'Yoruba',
 'Bamidele': 'Yoruba',
 'Bilƙisu': 'Hausa',
 'Bolanle': 'Yoruba',
 'Bose': 'Yoruba',
 'Bosede': 'Yoruba',
 'Bukola': 'Yoruba',
 'Chi': 'Igbo',
 'Chiamaka': 'Igbo',
 'Chibuzo': 'Igbo',
 'Chichi': 'Igbo',
 'Chidi': 'Igbo',
 'Chidiebele': 'Igbo',
 'Chidiebere'

In [4]:
import random
import pandas as pd
from faker import Faker
import numpy as np
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()
Faker.seed(42)
np.random.seed(42)

# Define constants
regions = ['North Central', 'North East', 'North West', 'South East', 'South South', 'South West']
nigeria_states = [
    "Abia", "Adamawa", "Akwa Ibom", "Anambra", "Bauchi", "Bayelsa", "Benue",
    "Borno", "Cross River", "Delta", "Ebonyi", "Edo", "Ekiti", "Enugu", "Gombe",
    "Imo", "Jigawa", "Kaduna", "Kano", "Katsina", "Kebbi", "Kogi", "Kwara",
    "Lagos", "Nasarawa", "Niger", "Ogun", "Ondo", "Osun", "Oyo", "Plateau",
    "Rivers", "Sokoto", "Taraba", "Yobe", "Zamfara", "FCT"
]



education_levels = ['None', 'Primary', 'Secondary', 'Tertiary']
occupations = ['Farmer', 'Trader', 'Teacher', 'Civil Servant', 'Engineer', 'Unemployed', 'Doctor', 'Nurse']
extracurricular_activities = ['Sports', 'Drama', 'Debate Club', 'Art','Jet club','Press club','Literature club']
common_subjects = ['Mathematics', 'English Language', 'Civic Education', 'Economics', 'CRS/Islam']
science_subjects = ['Physics', 'Chemistry', 'Biology', 'Geography', 'Computer Science']
art_subjects = ['Government', 'Commerce', 'Literature','History', 'Accounting']
subjects = common_subjects + science_subjects + art_subjects
health_condition = ['Asthma'] * 3 + ['Sickle Cell'] * 2 + ['Ulcer'] * 3 + ['Epilepsy']*1 + ['Dyslexia']*20 + ['None']*91
teacher_type =  ['Corper','Regular']
weights = [0.2, 0.8]  

# Define staff positions with number of positions and pay grade
staff_positions = {
    "Teacher": {"count": len(subjects), "pay_grade": 100000},
    "Principal": {"count": 1, "pay_grade": 170000},
    "Vice Principal": {"count": 1, "pay_grade": 150000},
    "Librarian": {"count": 2, "pay_grade": 80000},
    "School Nurse": {"count": 2, "pay_grade": 120000},
    "Administrative Assistant": {"count": 3, "pay_grade": 80000},
    "Cleaner": {"count": 3, "pay_grade": 30000},
    "Vendor": {"count": 2, "pay_grade": 40000},
    "Bus Driver": {"count": 2, "pay_grade": 70000},
    "Lab attendant": {"count": 2, "pay_grade": 90000},
    "Security Guard": {"count": 2, "pay_grade": 70000}
}

def random_dobs(min_age, max_age):
    now = datetime.now()
    return (now - timedelta(days=random.randint(min_age*365, max_age*365))).strftime("%Y-%m-%d")


def assign_scores(subjects):
    return {subject: random.randint(0, 100) for subject in subjects}




def generate_dim_class_resources():
    data = []
    
    # Class levels and segments
    levels = ['SS1', 'SS2', 'SS3']
    segments = ['A', 'B', 'C', 'D', 'E', 'F']
    
    for level in levels:
        for segment in segments:
            class_name = f"{level} Class {segment}"
            num_students = random.randint(50, 60)  # Number of students per class
            num_teachers = random.randint(5, 10)  # Number of teachers per class
            
            # Weekly hours and classroom resources
            weekly_teaching_hours = random.randint(20, 35)  # Weekly teaching hours
            weekly_library_time = random.randint(1, 5)  # Weekly library hours
            weekly_computer_training_time = random.randint(2, 3)  # Computer training hours
            weekly_lab_hours = random.randint(0, 4)  # Weekly lab hours
            
            # Classroom resources (boolean values)
            chalkboard = random.randint(1, 3)
            basic_textbooks = random.randint(0, 4)
            chairs_desks = num_students
            functional_fans = random.randint(0, 4)
            
            # Append class information to the list
            data.append({
                'Class_ID': class_name,
                'Number_of_Students': num_students,
                'Number_of_Teachers': num_teachers,
                'Weekly_Teaching_Hours': weekly_teaching_hours,
                'Weekly_Library_Time': weekly_library_time,
                'Weekly_Computer_Training_Time': weekly_computer_training_time,
                'Weekly_Lab_Hours': weekly_lab_hours,
                'Chalkboard': chalkboard,
                'Basic_Textbooks': basic_textbooks,
                'Chairs_Desks': chairs_desks,
                'Functional_Fans': functional_fans
            })
    
    return pd.DataFrame(data)



dim_class_resources = generate_dim_class_resources()

# Number of records
NUM_STUDENTS = sum(dim_class_resources.Number_of_Students)




# Generate staff data
def generate_staff_data():
    data = []
    for position, details in staff_positions.items():
        for _ in range(details["count"]):
            staff_member = {
                'Staff_ID': fake.unique.uuid4(),
                "Name": ', '.join(random.sample(list(name_ethnic_male.keys()), 2)),
                'Gender': random.choice(['Male', 'Female']),
                "Position": position,
                "Monthly Pay": details["pay_grade"],
                "Years of Experience": random.randint(0, 30),
                "Education Level": fake.random_element(elements=("High School", "Associate's", "Bachelor's", "Master's", "PhD")),
                "Date of Hire": fake.date_between(start_date="-30y", end_date="today"),
                "Full-time": fake.boolean(chance_of_getting_true=80)
            }
            data.append(staff_member)
    return data

# Generate staff data and create DataFrame
staff_data = generate_staff_data()
staff_table = pd.DataFrame(staff_data)
staff_table['Staff_ID'] = staff_table['Staff_ID'].str.replace('-', '', regex=False)

# Generate Dim Teacher
def generate_teacher_table():
    teachers_table = staff_table[staff_table['Position'] == 'Teacher'][['Staff_ID', 'Name']].copy()
    teachers_table['Teacher_ID'] = [fake.unique.uuid4() for i in range(len(teachers_table))]
    teachers_table['Teacher_ID'] = teachers_table['Teacher_ID'].str.replace('-', '', regex=False)
    teachers_table['Teacher Type'] = random.choices(teacher_type, weights=weights, k=len(teachers_table))
    teachers_table['Subject specialization'] = subjects
    teachers_table = teachers_table[['Teacher_ID', 'Staff_ID', 'Name', 'Teacher Type', 'Subject specialization']]
    return teachers_table

teachers_table = generate_teacher_table()




# # Generate Dim Event
# def generate_dim_event(num_events, dim_class):
#     data = []
#     for _ in range(num_events):
#         event_id = fake.unique.uuid4()
#         event_name = random.choice(['Sports Day', 'Annual Meeting', 'Science Fair', 'Art Exhibition', 'Debate Competition'])
#         event_type = random.choice(['Academic', 'Sports', 'Cultural'])
#         date = fake.date_between(start_date='-3y', end_date='today')
#         class_id = random.choice(dim_class['Class_Name'].tolist())
#         data.append({
#             'Event_ID': event_id,
#             'Event_Name': event_name,
#             'Event_Type': event_type,
#             'Date': date,
#             'class_ID': class_id
#         })
#     return pd.DataFrame(data)

# dim_event = generate_dim_event(NUM_EVENTS, dim_class)

# Generate Dim Student
def generate_dim_student(num_students, dim_class):
    data = []
    for _ in range(num_students):
        student_id = fake.unique.uuid4()
        gender = random.choice(['Male', 'Female'])
        DOB = random_dobs(14, 18) 
        region = random.choice(regions)
        class_id = random.choice(dim_class['Class_ID'].tolist())
        data.append({
            'Student_ID': student_id,
            'Class_ID': class_id,
            'First_Name': ', '.join(random.sample(list(name_ethnic_male.keys()), 1)),
            'Family_Name': ', '.join(random.sample(list(name_ethnic_male.keys()), 1)),
            'Gender': gender,
            'Date_of_Birth': DOB,
            'State of Origin': random.choice(nigeria_states),
            'engagement_in_class': random.choice(['Troublesome','Unactive','Slightly active','Active','Highly active']),
            'health_condition': random.choice(health_condition),
            'Class Spec': random.choice(['Art','Science'])
        })
    return pd.DataFrame(data)

dim_student = generate_dim_student(NUM_STUDENTS, dim_class_resources)
dim_student['Student_ID'] = dim_student['Student_ID'].str.replace('-', '', regex=False)

# Generate Dim Parent Demographics
def generate_dim_parent_demographics(dim_student):
    data = []
    for _, student in dim_student.iterrows():
        father_education = random.choice(education_levels)
        mother_education = random.choice(education_levels)
        father_occupation = random.choice(occupations)
        mother_occupation = random.choice(occupations)
        data.append({
            'Student_ID': student['Student_ID'],
            'Fathers Name': ', '.join(random.sample(list(name_ethnic_male.keys()), 1)),
            'Mothers Name': ', '.join(random.sample(list(name_ethnic_female.keys()), 1)),
            'Family Name' : ', '.join(random.sample(list(name_ethnic_male.keys()), 1)),
            'Father_Education': father_education,
            'Mother_Education': mother_education,
            'Father_Occupation': father_occupation,
            'Mother_Occupation': mother_occupation,
            'Annual_Household_Income(NGN)': random.choice(['Below 200,000', '200,000-400,000', '400,000-600,000', 'Above 600,000']),
            'Household_Size': random.choice(np.arange(2, 7)),
            'Involvement_in_Kids_Education': random.choice(['Always busy', 'Slightly involved', 'Involved', 'Very Involved'])
        })
    return pd.DataFrame(data)

dim_parent_demographics = generate_dim_parent_demographics(dim_student)

# Generate Dim Extracurricular Activity
def generate_dim_extracurricular_activity(dim_student):
    data = []
    for _, student in dim_student.iterrows():
        extracurricular = random.choice(extracurricular_activities)
        weekly_hours_in_activity = random.randint(1, 10) if extracurricular != 'None' else 0
        data.append({
            'Student_ID': student['Student_ID'],
            'Extracurricular_Activity': extracurricular,
            'Weekly_Hours': weekly_hours_in_activity
        })
    return pd.DataFrame(data)

dim_extracurricular_activity = generate_dim_extracurricular_activity(dim_student)


# Generate Fact School Operations
# def generate_fact_school_operations(dim_class):
#     data = []
#     for _, row in dim_class.iterrows():# Iterate over the rows properly
#         # Each class can have operational records
#         budget = random.randint(500000, 5000000)  # Annual budget in Naira
#         spending_on_library = random.randint(100000, 1000000)
#         spending_on_teachers = random.randint(200000, 2000000)
#         number_of_classrooms = len(dim_class)  # Use the total number of classrooms
#         library_books = random.randint(500, 10000)
#         health_staff = random.randint(1, 10)

#         # Append the data for each class
#         data.append({
#             'Class_ID': row['Class_Name'],  # Correctly assign the class name
#             'Budget': budget,
#             'Spending_On_Library': spending_on_library,
#             'Spending_On_Teachers': spending_on_teachers,
#             'Number_of_Classrooms': number_of_classrooms,
#             'Library_Books': library_books,
#             'Health_Staff': health_staff
#         })
    
#     return pd.DataFrame(data)

# fact_school_operations = generate_fact_school_operations(dim_class_resources)



def generate_fact_attendance(dim_student): 
    data = []
    
    # Randomly select a subset of students to miss school (between 10 and 50 students)
    students_missing_school = random.sample(dim_student.index.tolist(), random.randint(10, 50))
    
    for idx, student in dim_student.iterrows():
        if idx in students_missing_school:
            # Students who miss school
            days_attended = random.randint(60, 94)  # Randomly decide attended days between 80 and 94
            days_missed = 95 - days_attended
            absence_reason = random.choice(['Illness', 'Family Event', 'Other','Truancy','School fees drive','Insecurity'])
        else:
            # Students who don't miss school
            days_attended = 95
            days_missed = 0
            absence_reason = 'Full Attendance'
        
        data.append({
            'Student_ID': student['Student_ID'],
            'Days_Attended': days_attended,
            'Days_Missed': days_missed,
            'Absence_Reason': absence_reason
        })
        
    return pd.DataFrame(data)


# Generate attendance fact data
fact_attendance = generate_fact_attendance(dim_student)



def generate_student_performance(dim_student):
    dim_student_copy = dim_student[['Student_ID', 'Class Spec']].copy()  # Copy to avoid altering original DataFrame
    data = []
    
    # Iterate over each row in the DataFrame
    for _, row in dim_student_copy.iterrows():
        student_id = row['Student_ID']
        class_spec = row['Class Spec']
        
        # Common subjects (shared between both Art and Science students)
        performance = assign_scores(common_subjects)
        
        # Additional subjects depending on the stream
        if class_spec == 'Science':
            performance.update(assign_scores(science_subjects))
            # Art subjects will have None for Science students
            performance.update({subject: None for subject in art_subjects})
        else:
            performance.update(assign_scores(art_subjects))
            # Science subjects will have None for Art students
            performance.update({subject: None for subject in science_subjects})
        
        # Append student data to list
        data.append({
            'Student_ID': student_id,
            **performance
        })
    
    return pd.DataFrame(data)


# Generate student performance data
student_performance = generate_student_performance(dim_student)





# Assuming dim_student contains all students with their respective class information
def generate_ss3_student_performance_survey(dim_student):
    # Filter students from SS3A to SS3F
    ss3_students = dim_student[dim_student['Class_ID'].str.startswith('SS3')]

    # List of reasons for performance issues
    reasons_for_performance = ['Lack of preparation', 'Difficulty understanding topics', 
                               'Personal issues', 'Health challenges', 'Confidence issues', 
                               'Lack of resources']

    data = []
    for _, student in ss3_students.iterrows():
        # Generate random survey responses
        reason = random.choice(reasons_for_performance)
        access_to_resources = random.choice(['Yes', 'No'])
        study_hours_per_week = random.randint(0, 40)  # Study hours ranging from 0 to 40
        health_issues = random.choice(['Yes', 'No'])
        teacher_support = random.randint(1, 5)  # Rating from 1 to 5
        parental_support = random.randint(1, 5)  # Rating from 1 to 5
        stress_level = random.choice(['Yes', 'No'])
        peer_influence = random.choice(['Yes', 'No'])
        additional_tutoring = random.choice(['Yes', 'No'])
        exam_anxiety = random.choice(['Yes', 'No'])
        use_of_study_groups = random.choice(['Yes', 'No'])
                # JAMB Mock Scores and Credit Passes in WAEC
        jamb_mock_scores = random.randint(100, 400)
        num_credit_passes_waec = random.randint(2, 9)

        # Append the survey data for each student
        data.append({
            'Student_ID': student['Student_ID'],
            'Reason_For_Performance': reason,
            'Access_To_Resources': access_to_resources,
            'Study_Hours_Per_Week': study_hours_per_week,
            'Health_Issues': health_issues,
            'Teacher_Support': teacher_support,
            'Parental_Support': parental_support,
            'Stress_Level': stress_level,
            'Peer_Influence': peer_influence,
            'Additional_Tutoring': additional_tutoring,
            'Use_Of_Study_Groups': use_of_study_groups,
            'Exam_Anxiety': exam_anxiety,
            'Jamb_Scores': jamb_mock_scores,
            'Num_Credit_Passes_WAEC': num_credit_passes_waec
            
        })
    
    # Create a DataFrame from the generated data
    return pd.DataFrame(data)


# Generate the SS3 student performance survey table
ss3_student_survey = generate_ss3_student_performance_survey(dim_student)
ss3_student_survey['verdict'] = ss3_student_survey.apply(lambda row: 'Pass' if row['Jamb_Scores'] >= 200 and row['Num_Credit_Passes_WAEC'] >= 5 else 'Fail', axis=1)


# def art_science_grades():
#         # Example: Predefined high school art and science courses
#     art_courses = ['Government', 'Literature', 'Economics', 'CRK/Islam']
#     science_courses = ['Biology', 'Physics', 'Chemistry', 'Intro. Tech']

#     # Function to calculate rank based on score
#     def calculate_rank(score):
#         if score >= 70:
#             return 'A'
#         elif score >= 60:
#             return 'B'
#         elif score >= 50:
#             return 'C'
#         elif score >= 40:
#             return 'D'
#         else:
#             return 'F'

#     # Example DataFrame: dim_student with a 'student_id' and 'Class Spec' column
#     dim_student_copy = dim_student[['Student_ID','Class Spec']]

#     # Create art_discipline and science_discipline DataFrames based on the 'Class Spec' column
#     art_discipline = dim_student_copy[dim_student_copy['Class Spec'] == 'Art'].copy()
#     science_discipline = dim_student_copy[dim_student_copy['Class Spec'] == 'Science'].copy()

#     # Assign scores for art and science students
#     for course in art_courses:
#         art_discipline[course] = art_discipline.apply(lambda x: random.randint(50, 100), axis=1)

#     for course in science_courses:
#         science_discipline[course] = science_discipline.apply(lambda x: random.randint(50, 100), axis=1)

#     # Calculate average score for each student in art_discipline and science_discipline
#     art_discipline['Average_Score'] = art_discipline[art_courses].mean(axis=1)
#     science_discipline['Average_Score'] = science_discipline[science_courses].mean(axis=1)

#     # Calculate ranks based on the average score
#     art_discipline['Grade'] = art_discipline['Average_Score'].apply(calculate_rank)
#     science_discipline['Grade'] = science_discipline['Average_Score'].apply(calculate_rank)

    
#     art_discipline = art_discipline[['Student_ID'] + art_courses + ['Average_Score', 'Grade']]

    
#     science_discipline = science_discipline[['Student_ID'] + science_courses + ['Average_Score', 'Grade']]

#     return art_discipline, science_discipline

# art_scores_2021 ,science_scores_2021 = art_science_grades()
# art_scores_2022  ,science_scores_2022  = art_science_grades()
# art_scores_2023  ,science_scores_2023  = art_science_grades()



# Save all tables to CSV files
dim_class_resources
dim_student
dim_parent_demographics
dim_extracurricular_activity
teachers_table
student_performance
fact_attendance
staff_table
ss3_student_survey

Student_ID           Reason_For_Performance  \
0    310c0c003fa741049bf90e27dc96925e                Confidence issues   
1    23e2fcb472d8467d894a05e430b187ef                  Personal issues   
2    766ecb15474e4c19aef912766c006f61                Lack of resources   
3    db20a56edc814fe78eda8bbb71710434                Confidence issues   
4    03c72ba8d60547708a63f881ffd0f9d5                Confidence issues   
..                                ...                              ...   
309  49646b96fa3c4628892621df465568b7                Health challenges   
310  f090f5a0ce614199ae3a0ba8ac8d6c7d              Lack of preparation   
311  409e7a8086c541d69ea9f50831227592  Difficulty understanding topics   
312  79db78628d7d4772a08e2f5c61177270                  Personal issues   
313  47c68688ecf34053af36b8cc13087974              Lack of preparation   

    Access_To_Resources  Study_Hours_Per_Week Health_Issues  Teacher_Support  \
0                   Yes                    38            No                1   
1                    No                    37           Yes                4   
2                    No                     9            No                2   
3                    No                     6            No                2   
4                    No                    30           Yes                2   
..                  ...                   ...           ...              ...   
309                 Yes                    40            No                2   
310                  No                    28           Yes                3   
311                 Yes                    24           Yes                3   
312                 Yes                    14           Yes                5   
313                  No                     9            No                2   

     Parental_Support Stress_Level Peer_Influence Additional_Tutoring  \
0                   3          Yes             No                 Yes   
1                   4          Yes             No                  No   
2                   2           No             No                 Yes   
3                   2          Yes            Yes                 Yes   
4                   3           No            Yes                 Yes   
..                ...          ...            ...                 ...   
309                 3          Yes             No                 Yes   
310                 5           No            Yes                 Yes   
311                 5           No             No                  No   
312                 2          Yes             No                 Yes   
313                 1          Yes            Yes                  No   

    Use_Of_Study_Groups Exam_Anxiety  Jamb_Scores  Num_Credit_Passes_WAEC  \
0                   Yes           No          200                       7   
1                   Yes           No          136                       4   
2                    No          Yes          108                       6   
3                   Yes          Yes          283                       9   
4                    No           No          248                       5   
..                  ...          ...          ...                     ...   
309                 Yes          Yes          399                       5   
310                  No           No          316                       8   
311                  No           No          186                       5   
312                  No          Yes          207                       6   
313                  No          Yes          148                       8   

    verdict  
0      Pass  
1      Fail  
2      Fail  
3      Pass  
4      Pass  
..      ...  
309    Pass  
310    Pass  
311    Fail  
312    Pass  
313    Fail  

[314 rows x 15 columns]

In [23]:
import pandas as pd

# Load the data
attendance_df = fact_attendance
student_df = dim_student
class_resources_df = dim_class_resources
extracurricular_df = dim_extracurricular_activity
parent_df = dim_parent_demographics
survey_df = ss3_student_survey
staff_df = staff_table
performance_df = student_performance
teachers_df = teachers_table

def attendance_quality_checks():
    # Define total number of school days
    total_school_days = 95

    # 1. Check for null values in all columns
    if attendance_df.isnull().values.any():
        print("Null values found in the dataframe:\n", attendance_df.isnull().sum())
    else:
        print("No null values found.")

    # 2. Ensure Student_ID is unique and matches with student_table
    if not attendance_df['Student_ID'].is_unique:
        print("Duplicate Student_IDs found.")
    else:
        print("Student_ID is unique.")

    unmatched_ids = attendance_df[~attendance_df['Student_ID'].isin(student_df['Student_ID'])]
    if not unmatched_ids.empty:
        print(f"Unmatched Student_IDs found:\n{unmatched_ids['Student_ID'].values}")
    else:
        print("All Student_IDs match with student_table.")

    # 3. Verify Days_Attended and Days_Missed are non-negative
    if (attendance_df['Days_Attended'] < 0).any() or (attendance_df['Days_Missed'] < 0).any():
        print("Found negative values in Days_Attended or Days_Missed.")
    else:
        print("Days_Attended and Days_Missed are non-negative.")

    # 4. Check if Days_Attended + Days_Missed equals total school days
    if (attendance_df['Days_Attended'] + attendance_df['Days_Missed'] != total_school_days).any():
        mismatches = attendance_df[attendance_df['Days_Attended'] + attendance_df['Days_Missed'] != total_school_days]
        print(f"Mismatches found:\n{mismatches[['Student_ID', 'Days_Attended', 'Days_Missed']]}")
    else:
        print("Days_Attended + Days_Missed matches total school days.")

    # 5. Ensure Absence_Reason is filled for all records where Days_Missed > 0
    missing_reasons = attendance_df[(attendance_df['Days_Missed'] > 0) & (attendance_df['Absence_Reason'].isnull())]
    if not missing_reasons.empty:
        print(f"Absence_Reason missing for records where Days_Missed > 0:\n{missing_reasons[['Student_ID', 'Days_Missed']]}")
    else:
        print("Absence_Reason is filled for all records where Days_Missed > 0.")

    # Save to parquet if all checks pass
    if (attendance_df.isnull().values.any() == False and
        attendance_df['Student_ID'].is_unique and
        unmatched_ids.empty and
        (attendance_df['Days_Attended'] >= 0).all() and
        (attendance_df['Days_Missed'] >= 0).all() and
        (attendance_df['Days_Attended'] + attendance_df['Days_Missed'] == total_school_days).all() and
        missing_reasons.empty):
        
        attendance_df.to_parquet('./passed_basic_quality_checks/attendance_table.parquet')
        print("All attendance data quality checks passed. Data saved as Parquet.")
    else:
        print("Data quality checks failed.")




def class_resources_quality_checks():
    # 1. Check for null values in all columns
    if class_resources_df.isnull().values.any():
        print("Null values found in the dataframe:\n", class_resources_df.isnull().sum())
    else:
        print("No null values found.")

    # 2. Ensure Class_ID is unique
    if not class_resources_df['Class_ID'].is_unique:
        print("Duplicate Class_IDs found.")
    else:
        print("Class_ID is unique.")

    # 3. Verify all numeric columns have non-negative values
    numeric_columns = ['Number_of_Students', 'Number_of_Teachers', 'Weekly_Teaching_Hours', 
                    'Weekly_Library_Time', 'Weekly_Computer_Training_Time', 'Weekly_Lab_Hours',
                    'Chalkboard', 'Basic_Textbooks', 'Chairs_Desks', 'Functional_Fans']

    if (class_resources_df[numeric_columns] < 0).any().any():
        print("Found negative values in numeric columns.")
    else:
        print("All numeric columns have non-negative values.")

    # 4. Check if Number_of_Students and Number_of_T are reasonable
    if class_resources_df['Number_of_Students'].max() > 100 or class_resources_df['Number_of_Teachers'].max() > 20:
        print("Unreasonable values in Number_of_Students or Number_of_Teachers.")
    else:
        print("Number_of_Students and Number_of_Teachers are within reasonable limits.")

    # 5. Ensure Weekly_Teaching_Hours is within a realistic range (e.g., 20-50 hours)
    if not class_resources_df['Weekly_Teaching_Hours'].between(20, 50).all():
        print("Weekly_Teaching_Hours not within realistic range (20-50 hours).")
    else:
        print("Weekly_Teaching_Hours is within realistic range.")



    # Save to parquet if all checks pass
    if (class_resources_df.isnull().values.any() == False and
        class_resources_df['Class_ID'].is_unique and
        (class_resources_df[numeric_columns] >= 0).all().all() and
        class_resources_df['Number_of_Students'].max() <= 100 and
        class_resources_df['Number_of_Teachers'].max() <= 20 and
        class_resources_df['Weekly_Teaching_Hours'].between(20, 50).all()):
        
        class_resources_df.to_parquet('./passed_basic_quality_checks/class_resources_table.parquet')
        print("All data quality checks passed. Data saved as Parquet.")
    else:
        print("Data quality checks failed.")




def extracurricular_activities_data_checks():
    # 1. Check for null values in all columns
    if extracurricular_df.isnull().values.any():
        print("Null values found in the dataframe:\n", extracurricular_df.isnull().sum())
    else:
        print("No null values found.")

    # 2. Ensure Student_ID exists in student_table
    missing_students = extracurricular_df[~extracurricular_df['Student_ID'].isin(student_df['Student_ID'])]
    if not missing_students.empty:
        print("Student_ID(s) missing in student_table:\n", missing_students['Student_ID'].unique())
    else:
        print("All Student_IDs exist in student_table.")

    # 3. Verify Weekly_Hours is non-negative and within a realistic range (0-20 hours)
    if not extracurricular_df['Weekly_Hours'].between(0, 20).all():
        print("Weekly_Hours not in the realistic range (0-20 hours) or contains negative values.")
    else:
        print("Weekly_Hours is within a realistic range (0-20 hours) and non-negative.")

    # Save to parquet if all checks pass
    if (extracurricular_df.isnull().values.any() == False and
        missing_students.empty and
        extracurricular_df['Weekly_Hours'].between(0, 20).all()):
        
        extracurricular_df.to_parquet('./passed_basic_quality_checks/extracurricular_activity.parquet')
        print("All data quality checks passed. Data saved as Parquet.")
    else:
        print("Data quality checks failed.")




def parent_data_quality_checks():
    # 1. Check for null values in all columns
    if parent_df.isnull().values.any():
        print("Null values found:\n", parent_df.isnull().sum())
    else:
        print("No null values found.")

    # 2. Ensure Student_ID is unique and matches with student_table
    duplicate_student_ids = parent_df['Student_ID'].duplicated().sum()
    if duplicate_student_ids > 0:
        print(f"Found {duplicate_student_ids} duplicate Student_IDs.")
    else:
        print("All Student_IDs are unique.")

    missing_students = parent_df[~parent_df['Student_ID'].isin(student_df['Student_ID'])]
    if not missing_students.empty:
        print("Student_ID(s) missing in student_table:\n", missing_students['Student_ID'].unique())
    else:
        print("All Student_IDs exist in student_table.")

    # 3. Verify Household_Size is positive and within a realistic range (e.g., 1-15)
    if not parent_df['Household_Size'].between(1, 15).all():
        print("Household_Size contains values outside the realistic range (1-15).")
    else:
        print("Household_Size is within the realistic range.")

    # 4. Check if Annual_Household_Income(NGN) is valid
    valid_income_ranges = ['Below 200,000', '200,000-400,000', '400,000-600,000', 'Above 600,000']
    if not parent_df['Annual_Household_Income(NGN)'].isin(valid_income_ranges).all():
        print("Invalid values found in Annual_Household_Income(NGN).")
    else:
        print("Annual_Household_Income(NGN) has valid values.")

    # 5. Ensure Father_Education, Mother_Education, Father_Occupation, Mother_Occupation, and Involvement_in_Kids_Education have consistent categories
    consistent_columns = ['Father_Education', 'Mother_Education', 'Father_Occupation', 'Mother_Occupation', 'Involvement_in_Kids_Education']
    for col in consistent_columns:
        print(f"Unique values in {col}:\n", parent_df[col].unique())

    # Save to parquet if all checks pass
    if (parent_df.isnull().values.any() == False and
        duplicate_student_ids == 0 and
        missing_students.empty and
        parent_df['Household_Size'].between(1, 15).all() and
        parent_df['Annual_Household_Income(NGN)'].isin(valid_income_ranges).all()):
        
        parent_df.to_parquet('./passed_basic_quality_checks/parent_table.parquet')
        print("All data quality checks passed. Data saved as Parquet.")
    else:
        print("Data quality checks failed.")



def survey_data_quality():
    # 1. Check for null values in all columns
    if survey_df.isnull().values.any():
        print("Null values found:\n", survey_df.isnull().sum())
    else:
        print("No null values found.")

    # 2. Ensure Student_ID is unique and matches with student_table
    if survey_df['Student_ID'].duplicated().any():
        print("Duplicate Student_IDs found.")
    else:
        print("All Student_IDs are unique.")

    missing_students_survey = survey_df[~survey_df['Student_ID'].isin(student_df['Student_ID'])]
    if not missing_students_survey.empty:
        print("Student_ID(s) missing in student_table:\n", missing_students_survey['Student_ID'].unique())
    else:
        print("All Student_IDs exist in student_table.")

    # 3. Verify Study_Hours_Per_Week is non-negative and within a realistic range (0-50)
    if not survey_df['Study_Hours_Per_Week'].between(0, 50).all():
        print("Study_Hours_Per_Week contains values outside the realistic range (0-50).")
    else:
        print("Study_Hours_Per_Week is within the realistic range.")

    # 4. Check if Teacher_Support and Parental_Support are within a specific range (1-5)
    support_cols = ['Teacher_Support', 'Parental_Support']
    for col in support_cols:
        if not survey_df[col].between(1, 5).all():
            print(f"{col} contains values outside the range 1-5.")
        else:
            print(f"{col} is within the range 1-5.")

    # 5. Ensure Stress_Level has consistent categories
    print("Unique values in Stress_Level:", survey_df['Stress_Level'].unique())

    # 6. Verify Jamb_Scores and Num_Credit_Passes_WAEC are within expected ranges
    if not survey_df['Jamb_Scores'].between(0, 400).all():
        print("Jamb_Scores contain values outside the expected range (0-400).")
    else:
        print("Jamb_Scores are within the expected range.")

    if not survey_df['Num_Credit_Passes_WAEC'].between(0, 9).all():
        print("Num_Credit_Passes_WAEC contains values outside the expected range (0-9).")
    else:
        print("Num_Credit_Passes_WAEC is within the expected range.")

    # 7. Check if verdict has consistent categories
    print("Unique values in verdict:", survey_df['verdict'].unique())

    # Save to parquet if all checks pass
    if (survey_df.isnull().values.any() == False and
        not survey_df['Student_ID'].duplicated().any() and
        missing_students_survey.empty and
        survey_df['Study_Hours_Per_Week'].between(0, 50).all() and
        survey_df[support_cols].apply(lambda x: x.between(1, 5).all()).all() and
        survey_df['Jamb_Scores'].between(0, 400).all() and
        survey_df['Num_Credit_Passes_WAEC'].between(0, 9).all()):
        
        survey_df.to_parquet('./passed_basic_quality_checks/ss3_student_survey.parquet')
        print("All data quality checks passed. Data saved as Parquet.")
    else:
        print("Data quality checks failed.")


def staff_data_quality():
    # 1. Check for null values in all columns
    if staff_df.isnull().values.any():
        print("Null values found:\n", staff_df.isnull().sum())
    else:
        print("No null values found.")

    # 2. Ensure Staff_ID is unique
    if staff_df['Staff_ID'].duplicated().any():
        print("Duplicate Staff_IDs found.")
    else:
        print("All Staff_IDs are unique.")

    # 3. Verify Monthly Pay and Years of Experience are non-negative
    if (staff_df[['Monthly Pay', 'Years of Experience']] < 0).any().any():
        print("Monthly Pay or Years of Experience contains negative values.")
    else:
        print("Monthly Pay and Years of Experience are non-negative.")

    # 4. Check if Date of Hire is in a consistent date format and not in the future
    staff_df['Date of Hire'] = pd.to_datetime(staff_df['Date of Hire'], errors='coerce')
    future_dates = staff_df[staff_df['Date of Hire'] > pd.Timestamp.now()]
    if not future_dates.empty:
        print("Future Date of Hire values found:\n", future_dates['Date of Hire'])
    else:
        print("All Date of Hire values are valid.")

    # 5. Ensure Gender, Position, and Education Level have consistent categories
    print("Unique values in Gender:", staff_df['Gender'].unique())
    print("Unique values in Position:", staff_df['Position'].unique())
    print("Unique values in Education Level:", staff_df['Education Level'].unique())

    # Save to parquet if all checks pass
    if (staff_df.isnull().values.any() == False and
        not staff_df['Staff_ID'].duplicated().any() and
        not (staff_df[['Monthly Pay', 'Years of Experience']] < 0).any().any() and
        future_dates.empty):
        
        staff_df.to_parquet('./passed_basic_quality_checks/staff_table.parquet')
        print("All data quality checks passed. Data saved as Parquet.")
    else:
        print("Data quality checks failed.")



def performance_data_quality():

    # 1. Check for null values in relevant columns
    if performance_df[['Mathematics', 'English Language', 'Civic Education', 'Economics', 'CRS/Islam']].isnull().any().any():
        print("Null values found in key columns:\n", performance_df[['Mathematics', 'English Language', 'Civic Education', 'Economics', 'CRS/Islam']].isnull().sum())
    else:
        print("No null values found in the relevant columns.")

    # 2. Ensure Student_ID is unique and matches with student_table
    if performance_df['Student_ID'].duplicated().any():
        print("Duplicate Student_IDs found.")
    else:
        print("All Student_IDs are unique.")

    # Check if Student_IDs exist in student_table
    missing_students_perf = performance_df[~performance_df['Student_ID'].isin(student_df['Student_ID'])]
    if not missing_students_perf.empty:
        print("Student_ID(s) missing in student_table:\n", missing_students_perf['Student_ID'].unique())
    else:
        print("All Student_IDs exist in student_table.")

    # 3. Verify that scores for 'Mathematics', 'English Language', 'Civic Education', 'Economics', 'CRS/Islam' are within the expected range (0-100)
    columns_to_check = ['Mathematics', 'English Language', 'Civic Education', 'Economics', 'CRS/Islam']
    valid_scores = performance_df[columns_to_check].apply(lambda x: x.between(0, 100) | x.isna()).all()
    if not valid_scores.all():
        print("Some scores are outside the expected range (0-100) in these columns:", valid_scores[~valid_scores].index.tolist())
    else:
        print("All non-null scores are within the expected range (0-100).")

    # 4. Check for any outliers in the scores (based on z-scores), ignoring NaN
    z_scores = (performance_df[columns_to_check] - performance_df[columns_to_check].mean()) / performance_df[columns_to_check].std()
    outliers = z_scores.abs() > 3  # Threshold for outliers (z-score > 3)
    if outliers.any().any():
        print("Outliers found in the following columns:\n", outliers.columns[outliers.any()])
    else:
        print("No significant outliers found.")

    # 5. Ignore NaN columns, ensure decimal places for float64 columns are consistent
    float_columns = performance_df[columns_to_check].select_dtypes(include=['float64']).columns
    if performance_df[float_columns].apply(lambda x: x.apply(lambda v: len(str(v).split('.')[-1]) if '.' in str(v) else 0).nunique() > 1).any():
        print("Inconsistent decimal places in float64 columns.")
    else:
        print("Decimal places are consistent in float64 columns.")

    # Save to parquet if all checks pass
    if (performance_df[['Mathematics', 'English Language', 'Civic Education', 'Economics', 'CRS/Islam']].isnull().any().any() == False and
        not performance_df['Student_ID'].duplicated().any() and
        missing_students_perf.empty and
        valid_scores.all() and
        not outliers.any().any()):


        
        performance_df.to_parquet('./passed_basic_quality_checks/student_performance.parquet')
        print("All data quality checks passed. Data saved as Parquet.")
    else:
        print("Data quality checks failed.")



def student_data_quality_checks():
    # 1. Check for null values in all columns
    if student_df.isnull().any().any():
        print("Null values found in the following columns:\n", student_df.isnull().sum())
    else:
        print("No null values found in any columns.")

    # 2. Ensure Student_ID is unique
    if student_df['Student_ID'].duplicated().any():
        print("Duplicate Student_IDs found.")
    else:
        print("All Student_IDs are unique.")

    # 3. Verify Class_ID exists in class_resources_table
    missing_class_ids = student_df[~student_df['Class_ID'].isin(class_resources_df['Class_ID'])]
    if not missing_class_ids.empty:
        print("Class_ID(s) missing in class_resources_table:\n", missing_class_ids['Class_ID'].unique())
    else:
        print("All Class_IDs exist in class_resources_table.")

    # 4. Check if Date_of_Birth is in a consistent date format and makes sense for a student
    # Assuming the format is 'YYYY-MM-DD'
    def check_date_format(date_str):
        try:
            return pd.to_datetime(date_str, format='%Y-%m-%d', errors='raise')
        except Exception:
            return None

    invalid_dates = student_df['Date_of_Birth'].apply(check_date_format).isnull()
    if invalid_dates.any():
        print("Invalid Date_of_Birth found:\n", student_df[invalid_dates]['Date_of_Birth'])
    else:
        print("All Date_of_Births are in a consistent format.")

    # 5. Ensure Gender, State of Origin, engagement_in_class, health_condition, and Class Spec have consistent categories
    # Example categories
    gender_categories = ['Male', 'Female']
    state_origin_categories = nigeria_states  
    engagement_categories = ['Troublesome','Unactive','Slightly active','Active','Highly active']
    health_condition_categories = health_condition
    class_spec_categories = ['Science', 'Art']  

    if not student_df['Gender'].isin(gender_categories).all():
        print("Inconsistent categories found in Gender column.")
    if not student_df['State of Origin'].isin(state_origin_categories).all():
        print("Inconsistent categories found in State of Origin column.")
    if not student_df['engagement_in_class'].isin(engagement_categories).all():
        print("Inconsistent categories found in engagement_in_class column.")
    if not student_df['health_condition'].isin(health_condition_categories).all():
        print("Inconsistent categories found in health_condition column.")
    if not student_df['Class Spec'].isin(class_spec_categories).all():
        print("Inconsistent categories found in Class Spec column.")

    # Save to parquet if all checks pass
    if (not student_df.isnull().any().any() and
        not student_df['Student_ID'].duplicated().any() and
        missing_class_ids.empty and not invalid_dates.any()):
        
        student_df.to_parquet('./passed_basic_quality_checks/student_table.parquet')
        print("All data quality checks passed. Data saved as Parquet.")
    else:
        print("Data quality checks failed.")



def teacher_data_quality_checks():

    # 1. Check for null values in all columns
    if teachers_df.isnull().any().any():
        print("Null values found in the following columns:\n", teachers_df.isnull().sum())
    else:
        print("No null values found in any columns.")

    # 2. Ensure Teacher_ID is unique
    if teachers_df['Teacher_ID'].duplicated().any():
        print("Duplicate Teacher_IDs found.")
    else:
        print("All Teacher_IDs are unique.")

    # 3. Verify Staff_ID exists in staff_table
    missing_staff_ids = teachers_df[~teachers_df['Staff_ID'].isin(staff_df['Staff_ID'])]
    if not missing_staff_ids.empty:
        print("Staff_ID(s) missing in staff_table:\n", missing_staff_ids['Staff_ID'].unique())
    else:
        print("All Staff_IDs exist in staff_table.")

    # 4. Ensure Teacher Type and Subject specialization have consistent categories
    # Example categories
    teacher_type_categories = ['Full-Time', 'Part-Time', 'Substitute']  # Update as per your data
    subject_specialization_categories = ['Math', 'Science', 'English', 'History']  # Update as per your data

    if not teachers_df['Teacher Type'].isin(teacher_type_categories).all():
        print("Inconsistent categories found in Teacher Type column.")
    if not teachers_df['Subject specialization'].isin(subject_specialization_categories).all():
        print("Inconsistent categories found in Subject specialization column.")

    # Save to parquet if all checks pass
    if (not teachers_df.isnull().any().any() and
        not teachers_df['Teacher_ID'].duplicated().any() and
        missing_staff_ids.empty):
        
        teachers_df.to_parquet('./passed_basic_quality_checks/teachers_table.parquet')
        print("All data quality checks passed. Data saved as Parquet.")
    else:
        print("Data quality checks failed.")

student_data_quality_checks()

performance_data_quality()

staff_data_quality()

survey_data_quality()

parent_data_quality_checks()

extracurricular_activities_data_checks()

class_resources_quality_checks()

attendance_quality_checks()


No null values found in any columns.
All Student_IDs are unique.
All Class_IDs exist in class_resources_table.
All Date_of_Births are in a consistent format.
All data quality checks passed. Data saved as Parquet.
